In [1040]:
import numpy as np
import pandas as pd
import os
import re

In [1041]:
category = {
"Balance_Sheet" : [
                "Total Share Capital",
                "Reserves and Surplus",
                "Total Reserves and Surplus",
                "Total Shareholders Funds",
                "Other Current Liabilities",
                "Total Current Liabilities",
                "Total Capital And Liabilities",
                "Tangible Assets",
                "Fixed Assets",
                "Total Non-Current Assets",
                "Total Current Assets",
                "Total Assets",
                ],

"Cash_Flow" : [
                "Net Profit/Loss Before Extraordinary Items And Tax",
                "Net CashFlow From Operating Activities",
                "Net Cash Used In Investing Activities",
                "Net Cash Used From Financing Activities",
                "Foreign Exchange Gains / Losses",
                "Net Inc/Dec In Cash And Cash Equivalents",
                "Cash And Cash Equivalents Begin of Year",
                "Cash And Cash Equivalents End Of Year",
                ],

"Profit_Loss" : [ 
                "Revenue From Operations [Gross]",
                "Revenue From Operations [Net]",
                "Total Operating Revenues",
                "Other Income",
                "Total Revenue",
                "Operating And Direct Expenses",
                "Employee Benefit Expenses",
                "Finance Costs",
                "Depreciation And Amortisation Expenses",
                "Other Expenses",
                "Total Expenses",
                "Profit/Loss Before Exceptional, ExtraOrdinary Items And Tax",
                "Profit/Loss Before Tax",
                "Current Tax",
                "Deferred Tax",
                "Total Tax Expenses",
                "Profit/Loss After Tax And Before ExtraOrdinary Items",
                "Profit/Loss From Continuing Operations",
                "Profit/Loss For The Period",
                "Basic EPS (Rs.)",
                "Diluted EPS (Rs.)",
               ],

"Quarterly" : [        
                "Net Sales/Income from operations",
                "Total Income From Operations",
                "Employees Cost",
                "depreciat",
                "Other Expenses",
                "P/L Before Other Inc. , Int., Excpt. Items & Tax",
                "Other Income",
                "P/L Before Int., Excpt. Items & Tax",
                "Interest",
                "P/L Before Exceptional Items & Tax",
                "P/L Before Tax",
                "Tax",
                "P/L After Tax from Ordinary Activities",
                "Net Profit/(Loss) For the Period",
                "Equity Share Capital",
                "Basic EPS",
                "Diluted EPS",
                "Basic EPS.",
                "Diluted EPS.",
        ],

"Ratio" : [        
                "Revenue from Operations/Share (Rs.)",
                "PBDIT/Share (Rs.)",
                "PBIT/Share (Rs.)",
                "PBT/Share (Rs.)",
                "Net Profit/Share (Rs.)",
                "PBDIT Margin (%)",
                "PBIT Margin (%)",
                "PBT Margin (%)",
                "Net Profit Margin (%)",
                "Return on Networth / Equity (%)",
                "Return on Assets (%)",
                "Total Debt/Equity (X)",
                "Dividend Payout Ratio (NP) (%)",
                "Dividend Payout Ratio (CP) (%)",
                "Earnings Retention Ratio (%)",
                "Enterprise Value (Cr.)",
                "EV/EBITDA (X)",
        ],

"Year" : [        
                "Net Sales/Income from operations",
                "Total Income From Operations",
                "Employees Cost",
                "Depreciat",
                "Other Expenses",
                "P/L Before Other Inc. , Int., Excpt. Items & Tax",
                "Other Income",
                "P/L Before Int., Excpt. Items & Tax",
                "Interest",
                "P/L Before Exceptional Items & Tax",
                "P/L Before Tax",
                "Tax",
                "P/L After Tax from Ordinary Activities",
                "Net Profit/(Loss) For the Period",
                "Equity Share Capital",
                "Basic EPS",
                "Diluted EPS",
                "Basic EPS.",
                "Diluted EPS.",
        ]
}

In [1042]:
input_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\3i Infotech Ltd\Excel\Ratios_combined.xlsx"
output_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Data Preporcessing\3i infotech\Ratios_cleaned.xlsx"

In [1043]:
df = pd.read_excel(input_file)
df.head(3)

,Key Financial Ratios of 3i Infotech(in Rs. Cr.),Mar 09,Mar 08,Mar 07,Mar 06,Mar 05,Mar 14,Mar 13,Mar 12,Mar 11,...,Mar 19,Mar 18,Mar 17,Mar 16,Mar 15,Mar 24,Mar 23,Mar 22,Mar 21,Mar 20
0,Per Share Ratios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Basic EPS (Rs.),14.12,7.32,11.41,7.41,5.68,-7.78,-5.92,-17.37,5.85,...,1.55,-0.54,0.75,-8.67,-18.25,-48.52,3.10,0.5,2.01,0.79
2,Diluted EPS (Rs.),14.12,7.32,11.41,7.41,5.68,-7.78,-5.92,-17.37,5.83,...,1.55,-0.54,0.75,-8.67,-18.25,-48.52,3.05,0.5,2.01,0.79


In [1044]:
df.iloc[:,0]

0                              Per Share Ratios
1                               Basic EPS (Rs.)
2                             Diluted EPS (Rs.)
3                                Cash EPS (Rs.)
4     Book Value [ExclRevalReserve]/Share (Rs.)
5     Book Value [InclRevalReserve]/Share (Rs.)
6                         Dividend / Share(Rs.)
7           Revenue from Operations/Share (Rs.)
8                             PBDIT/Share (Rs.)
9                              PBIT/Share (Rs.)
10                              PBT/Share (Rs.)
11                       Net Profit/Share (Rs.)
12                         Profitability Ratios
13                             PBDIT Margin (%)
14                              PBIT Margin (%)
15                               PBT Margin (%)
16                        Net Profit Margin (%)
17              Return on Networth / Equity (%)
18               Return on Capital Employed (%)
19                         Return on Assets (%)
20                        Total Debt/Equ

In [1045]:

# Extract df_parameters column
df_parameters = df.iloc[:, 0].astype(str)  # Convert to string to avoid NaN issues
df_parameters = list(df_parameters)

In [1046]:
print(df_parameters)
print(len(df_parameters))

['Per Share Ratios', 'Basic EPS (Rs.)', 'Diluted EPS (Rs.)', 'Cash EPS (Rs.)', 'Book Value [ExclRevalReserve]/Share (Rs.)', 'Book Value [InclRevalReserve]/Share (Rs.)', 'Dividend / Share(Rs.)', 'Revenue from Operations/Share (Rs.)', 'PBDIT/Share (Rs.)', 'PBIT/Share (Rs.)', 'PBT/Share (Rs.)', 'Net Profit/Share (Rs.)', 'Profitability Ratios', 'PBDIT Margin (%)', 'PBIT Margin (%)', 'PBT Margin (%)', 'Net Profit Margin (%)', 'Return on Networth / Equity (%)', 'Return on Capital Employed (%)', 'Return on Assets (%)', 'Total Debt/Equity (X)', 'Asset Turnover Ratio (%)', 'Liquidity Ratios', 'Current Ratio (X)', 'Quick Ratio (X)', 'Inventory Turnover Ratio (X)', 'Dividend Payout Ratio (NP) (%)', 'Dividend Payout Ratio (CP) (%)', 'Earnings Retention Ratio (%)', 'Cash Earnings Retention Ratio (%)', 'Valuation Ratios', 'Enterprise Value (Cr.)', 'EV/Net Operating Revenue (X)', 'EV/EBITDA (X)', 'MarketCap/Net Operating Revenue (X)', 'Retention Ratios (%)', 'Price/BV (X)', 'Price/Net Operating Reven

In [1047]:
print(category["Ratio"])
print(len(category["Ratio"]))

['Revenue from Operations/Share (Rs.)', 'PBDIT/Share (Rs.)', 'PBIT/Share (Rs.)', 'PBT/Share (Rs.)', 'Net Profit/Share (Rs.)', 'PBDIT Margin (%)', 'PBIT Margin (%)', 'PBT Margin (%)', 'Net Profit Margin (%)', 'Return on Networth / Equity (%)', 'Return on Assets (%)', 'Total Debt/Equity (X)', 'Dividend Payout Ratio (NP) (%)', 'Dividend Payout Ratio (CP) (%)', 'Earnings Retention Ratio (%)', 'Enterprise Value (Cr.)', 'EV/EBITDA (X)']
17


In [1048]:
not_common_parameters = []

for item in df_parameters:
    if item not in category["Ratio"]:
        not_common_parameters.append(item)
        
print(not_common_parameters)
print(len(not_common_parameters))

['Per Share Ratios', 'Basic EPS (Rs.)', 'Diluted EPS (Rs.)', 'Cash EPS (Rs.)', 'Book Value [ExclRevalReserve]/Share (Rs.)', 'Book Value [InclRevalReserve]/Share (Rs.)', 'Dividend / Share(Rs.)', 'Profitability Ratios', 'Return on Capital Employed (%)', 'Asset Turnover Ratio (%)', 'Liquidity Ratios', 'Current Ratio (X)', 'Quick Ratio (X)', 'Inventory Turnover Ratio (X)', 'Cash Earnings Retention Ratio (%)', 'Valuation Ratios', 'EV/Net Operating Revenue (X)', 'MarketCap/Net Operating Revenue (X)', 'Retention Ratios (%)', 'Price/BV (X)', 'Price/Net Operating Revenue', 'Earnings Yield']
22


In [1049]:
index_to_drop =df[df.iloc[:, 0].isin(not_common_parameters)].index

In [1050]:
df = df.drop(index_to_drop,axis=0)

In [1051]:
df.head(5)

,Key Financial Ratios of 3i Infotech(in Rs. Cr.),Mar 09,Mar 08,Mar 07,Mar 06,Mar 05,Mar 14,Mar 13,Mar 12,Mar 11,...,Mar 19,Mar 18,Mar 17,Mar 16,Mar 15,Mar 24,Mar 23,Mar 22,Mar 21,Mar 20
7,Revenue from Operations/Share (Rs.),40.32,34.26,59.04,52.04,66.78,5.70,6.39,26.78,28.66,...,1.76,1.47,2.36,5.39,6.64,21.12,16.40,12.63,1.29,1.22
8,PBDIT/Share (Rs.),20.74,12.41,14.42,13.45,14.99,2.07,2.26,6.82,15.28,...,2.09,0.02,1.57,2.32,1.23,-21.30,4.03,3.17,0.37,0.98
9,PBIT/Share (Rs.),17.07,10.49,11.97,8.98,9.51,-2.06,-1.67,2.13,12.58,...,2.05,-0.02,1.51,-0.97,-2.47,-22.77,2.78,2.42,0.29,0.88
10,PBT/Share (Rs.),12.34,7.59,8.70,7.73,6.06,-7.92,-4.31,-13.15,5.63,...,1.55,-0.54,0.77,-6.99,-17.13,-48.37,3.10,0.50,1.57,0.34
11,Net Profit/Share (Rs.),14.12,7.32,11.41,7.41,5.68,-7.69,-4.47,-16.99,6.22,...,1.55,-0.54,0.78,-8.42,-17.52,-48.37,3.10,0.50,2.01,0.79


In [1052]:
df.replace("12 mths",np.nan,inplace=True)
df.head(3)

,Key Financial Ratios of 3i Infotech(in Rs. Cr.),Mar 09,Mar 08,Mar 07,Mar 06,Mar 05,Mar 14,Mar 13,Mar 12,Mar 11,...,Mar 19,Mar 18,Mar 17,Mar 16,Mar 15,Mar 24,Mar 23,Mar 22,Mar 21,Mar 20
7,Revenue from Operations/Share (Rs.),40.32,34.26,59.04,52.04,66.78,5.70,6.39,26.78,28.66,...,1.76,1.47,2.36,5.39,6.64,21.12,16.40,12.63,1.29,1.22
8,PBDIT/Share (Rs.),20.74,12.41,14.42,13.45,14.99,2.07,2.26,6.82,15.28,...,2.09,0.02,1.57,2.32,1.23,-21.30,4.03,3.17,0.37,0.98
9,PBIT/Share (Rs.),17.07,10.49,11.97,8.98,9.51,-2.06,-1.67,2.13,12.58,...,2.05,-0.02,1.51,-0.97,-2.47,-22.77,2.78,2.42,0.29,0.88


In [1053]:
df.dropna(inplace=True)
# df.reset_index(drop=True, inplace=True)

In [1054]:
df.iloc[:, 1:] = df.iloc[:, 1:].replace(",", "", regex=True)

In [1055]:
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

In [1056]:
column_start = df.columns[0]
column_start

'Key Financial Ratios of 3i Infotech(in Rs. Cr.)'

In [1057]:
df.columns[:]

Index(['Key Financial Ratios of 3i Infotech(in Rs. Cr.)', 'Mar 09', 'Mar 08',
       'Mar 07', 'Mar 06', 'Mar 05', 'Mar 14', 'Mar 13', 'Mar 12', 'Mar 11',
       'Mar 10', 'Mar 19', 'Mar 18', 'Mar 17', 'Mar 16', 'Mar 15', 'Mar 24',
       'Mar 23', 'Mar 22', 'Mar 21', 'Mar 20'],
      dtype='object')

In [1058]:
def extract_year(column_name):
    match = re.search(r'\d+', str(column_name))  # Extract the first number found
    return int(match.group()) if match else float('inf')  # Assign high value if no number


In [1059]:
# Sort columns by extracted year values
sorted_columns = sorted(df.columns[1:], key=extract_year)

# Reorder DataFrame columns
df = df[[column_start] + sorted_columns]

# Display the first few rows
df.head(3)

,Key Financial Ratios of 3i Infotech(in Rs. Cr.),Mar 05,Mar 06,Mar 07,Mar 08,Mar 09,Mar 10,Mar 11,Mar 12,Mar 13,...,Mar 15,Mar 16,Mar 17,Mar 18,Mar 19,Mar 20,Mar 21,Mar 22,Mar 23,Mar 24
7,Revenue from Operations/Share (Rs.),66.78,52.04,59.04,34.26,40.32,30.81,28.66,26.78,6.39,...,6.64,5.39,2.36,1.47,1.76,1.22,1.29,12.63,16.40,21.12
8,PBDIT/Share (Rs.),14.99,13.45,14.42,12.41,20.74,16.08,15.28,6.82,2.26,...,1.23,2.32,1.57,0.02,2.09,0.98,0.37,3.17,4.03,-21.30
9,PBIT/Share (Rs.),9.51,8.98,11.97,10.49,17.07,10.49,12.58,2.13,-1.67,...,-2.47,-0.97,1.51,-0.02,2.05,0.88,0.29,2.42,2.78,-22.77


In [1060]:
df = df.T.reset_index()
df.head(3)

,index,7,8,9,10,11,13,14,15,16,17,19,20,26,27,28,31,33
0,Key Financial Ratios of 3i Infotech(in Rs. Cr.),Revenue from Operations/Share (Rs.),PBDIT/Share (Rs.),PBIT/Share (Rs.),PBT/Share (Rs.),Net Profit/Share (Rs.),PBDIT Margin (%),PBIT Margin (%),PBT Margin (%),Net Profit Margin (%),Return on Networth / Equity (%),Return on Assets (%),Total Debt/Equity (X),Dividend Payout Ratio (NP) (%),Dividend Payout Ratio (CP) (%),Earnings Retention Ratio (%),Enterprise Value (Cr.),EV/EBITDA (X)
1,Mar 05,66.78,14.99,9.51,6.06,5.68,22.44,14.23,9.07,8.5,28.9,3.93,2.33,30.67,15.53,69.33,0.0,0.0
2,Mar 06,52.04,13.45,8.98,7.73,7.41,25.83,17.26,14.85,14.23,14.43,4.9,1.16,26.99,16.84,73.01,1134.78,15.91


In [1061]:
print(df.columns)
print(len(df.columns))

Index(['index', 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19, 20, 26, 27, 28, 31,
       33],
      dtype='object')
18


In [1062]:
df.to_excel(output_file)

In [1063]:
correct_headers =  [column_start] + category["Ratio"]
correct_headers

['Key Financial Ratios of 3i Infotech(in Rs. Cr.)',
 'Revenue from Operations/Share (Rs.)',
 'PBDIT/Share (Rs.)',
 'PBIT/Share (Rs.)',
 'PBT/Share (Rs.)',
 'Net Profit/Share (Rs.)',
 'PBDIT Margin (%)',
 'PBIT Margin (%)',
 'PBT Margin (%)',
 'Net Profit Margin (%)',
 'Return on Networth / Equity (%)',
 'Return on Assets (%)',
 'Total Debt/Equity (X)',
 'Dividend Payout Ratio (NP) (%)',
 'Dividend Payout Ratio (CP) (%)',
 'Earnings Retention Ratio (%)',
 'Enterprise Value (Cr.)',
 'EV/EBITDA (X)']

In [1064]:
print(correct_headers)
print(len(correct_headers))

['Key Financial Ratios of 3i Infotech(in Rs. Cr.)', 'Revenue from Operations/Share (Rs.)', 'PBDIT/Share (Rs.)', 'PBIT/Share (Rs.)', 'PBT/Share (Rs.)', 'Net Profit/Share (Rs.)', 'PBDIT Margin (%)', 'PBIT Margin (%)', 'PBT Margin (%)', 'Net Profit Margin (%)', 'Return on Networth / Equity (%)', 'Return on Assets (%)', 'Total Debt/Equity (X)', 'Dividend Payout Ratio (NP) (%)', 'Dividend Payout Ratio (CP) (%)', 'Earnings Retention Ratio (%)', 'Enterprise Value (Cr.)', 'EV/EBITDA (X)']
18


In [1065]:
# correct_headers

In [1066]:
# print(f"Number of columns in df: {len(df.columns)}")
# print(f"Number of headers in correct_headers: {len(correct_headers)}")


In [1067]:
# # Rename the DataFrame columns
# df.columns = correct_headers

In [1068]:
# df.to_excel(output_file,index=False)

In [1069]:
df.head(5)

,index,7,8,9,10,11,13,14,15,16,17,19,20,26,27,28,31,33
0,Key Financial Ratios of 3i Infotech(in Rs. Cr.),Revenue from Operations/Share (Rs.),PBDIT/Share (Rs.),PBIT/Share (Rs.),PBT/Share (Rs.),Net Profit/Share (Rs.),PBDIT Margin (%),PBIT Margin (%),PBT Margin (%),Net Profit Margin (%),Return on Networth / Equity (%),Return on Assets (%),Total Debt/Equity (X),Dividend Payout Ratio (NP) (%),Dividend Payout Ratio (CP) (%),Earnings Retention Ratio (%),Enterprise Value (Cr.),EV/EBITDA (X)
1,Mar 05,66.78,14.99,9.51,6.06,5.68,22.44,14.23,9.07,8.5,28.9,3.93,2.33,30.67,15.53,69.33,0.0,0.0
2,Mar 06,52.04,13.45,8.98,7.73,7.41,25.83,17.26,14.85,14.23,14.43,4.9,1.16,26.99,16.84,73.01,1134.78,15.91
3,Mar 07,59.04,14.42,11.97,8.7,11.41,24.41,20.27,14.73,19.33,17.76,5.67,1.51,17.52,14.42,82.48,1939.34,23.9
4,Mar 08,34.26,12.41,10.49,7.59,7.32,36.21,30.6,22.13,21.35,17.52,4.61,2.32,21.9,16.66,78.1,2593.85,16.01


In [1070]:
# if (df.iloc[0] == df.columns).all():
#     df = df[1:].reset_index(drop=True)  

In [1071]:
df.head(5)

,index,7,8,9,10,11,13,14,15,16,17,19,20,26,27,28,31,33
0,Key Financial Ratios of 3i Infotech(in Rs. Cr.),Revenue from Operations/Share (Rs.),PBDIT/Share (Rs.),PBIT/Share (Rs.),PBT/Share (Rs.),Net Profit/Share (Rs.),PBDIT Margin (%),PBIT Margin (%),PBT Margin (%),Net Profit Margin (%),Return on Networth / Equity (%),Return on Assets (%),Total Debt/Equity (X),Dividend Payout Ratio (NP) (%),Dividend Payout Ratio (CP) (%),Earnings Retention Ratio (%),Enterprise Value (Cr.),EV/EBITDA (X)
1,Mar 05,66.78,14.99,9.51,6.06,5.68,22.44,14.23,9.07,8.5,28.9,3.93,2.33,30.67,15.53,69.33,0.0,0.0
2,Mar 06,52.04,13.45,8.98,7.73,7.41,25.83,17.26,14.85,14.23,14.43,4.9,1.16,26.99,16.84,73.01,1134.78,15.91
3,Mar 07,59.04,14.42,11.97,8.7,11.41,24.41,20.27,14.73,19.33,17.76,5.67,1.51,17.52,14.42,82.48,1939.34,23.9
4,Mar 08,34.26,12.41,10.49,7.59,7.32,36.21,30.6,22.13,21.35,17.52,4.61,2.32,21.9,16.66,78.1,2593.85,16.01


In [1072]:
# output_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Data Preporcessing\3i infotech\cleanedDataBS_2.xlsx"
# df.to_excel(output_file,index=False)